### Video Processing

In [12]:
FRAMEWORK_PATH = "./static/results/framework/"
MIN_VIDEOS = 20

import json
import os

from preprocess import pre_process_videos

def get_muffin_video_transcripts():
    library_metadata = {}
    with open("./metadata.json") as f:
        library_metadata = json.load(f)

    task_metadata = library_metadata["muffins"]

    muffin_videos = pre_process_videos(task_metadata["videos"])
    
    transcripts = []
    for video in muffin_videos:
        url = f"https://www.youtube.com/watch?v={video.video_id}"
        title = video.metadata["title"]
        content = ""
        transcript = []
        for sentence in video.sentences:
            if sentence['text'].strip() == "":
                continue
            content += f"{sentence['text']}\n"
            transcript.append({
                "text": sentence['text'],
                "start": sentence['start'],
                "end": sentence['finish'],
            })

        transcripts.append({
            "url": url,
            "title": "Making Muffins",
            "original_title": title,
            "content": content,
            "transcript": transcript,
        })
    return transcripts

def get_muffin_articles():
    database_path = "./static/datasets/muffin_articles/"
    articles = []
    
    for filename in os.listdir(database_path):
        with open(database_path + filename) as f:
            ### read line-by-line
            url = f.readline()
            title = f.readline()
            content = ""
            transcript = []
            for idx, line in enumerate(f):
                if line.strip() == "":
                    continue
                content += line
                transcript.append({
                    "text": line.strip(),
                    "start": idx,
                    "end": idx + 1,
                })

            articles.append({
                "url": url,
                "original_title": title,
                "title": "Making Muffins",
                "content": content,
                "transcript": transcript,
            })
    return articles

def get_dataset_muffins(task, dummy=""):
    dataset_filepath = f"{FRAMEWORK_PATH}{task.replace(' ', '_').lower()}_{dummy}.json"
    if os.path.exists(dataset_filepath):
        with open(dataset_filepath) as f:
            dataset = json.load(f)
        return dataset

    dataset = get_muffin_articles()
    dataset = dataset + get_muffin_video_transcripts()
    print(f"Number of articles: {len(dataset)}")

    # dataset = add_info_labels_to_dataset(dataset, task)

    with open(dataset_filepath, "w") as f:
        json.dump(dataset, f, indent=4)
    return dataset

### Handle CrossTask data;
import csv
from helpers.video_scripts import extract_transcript

def library_cross_task():
    library = []
    PATH = "./static/datasets/crosstask/"
    library_path = os.path.join(PATH, "library.json")
    
    if os.path.exists(library_path):
        with open(library_path, "r") as f:
            library = json.load(f)
            return library

    tasks_path = os.path.join(PATH, "crosstask_release/tasks_primary.txt")
    videos_path = os.path.join(PATH, "crosstask_release/videos.csv")
    videos_val_path = os.path.join(PATH, "crosstask_release/videos_val.csv")

    """
    Task ID
    Task name
    URL of corresponding WikiHow page
    Number of steps
    Ordered list of comma-separated steps of the task
    """
    task_obj_ids = ["task_id", "task_name", "url", "num_steps", "steps"]

    with open(tasks_path) as f:
        lines = f.readlines()
        for start_idx in range(0, len(lines), 6):
            cur_task = {}
            finished = False
            for idx, task_obj_id in enumerate(task_obj_ids):
                if start_idx + idx >= len(lines):
                    finished = True
                    break
                cur_task[task_obj_id] = lines[start_idx + idx].strip()
            if finished is False:
                library.append(cur_task)

    for task in library:
        task["steps"] = task["steps"].split(",")
        task["videos"] = []

    for videos_path in [videos_path, videos_val_path]:
        with open(videos_path) as f:
            reader = csv.reader(f)
            for row in reader:
                task_id = row[0]
                video_id = row[1]
                video_url = row[2]
                for task in library:
                    if task["task_id"] == task_id:
                        task["videos"].append({
                            "video_id": video_id,
                            "video_url": video_url,
                        })

    def get_language(video_subtitles_path):
        with open(video_subtitles_path) as f:
            lines = f.readlines()
            for line in lines:
                if "Language:" in line:
                    return line.split(":")[1].strip()
        return None


    SUBTITLES_PATH = os.path.join(PATH, "subtitles")
    for task in library:
        for video in task["videos"]:
            video_id = video["video_id"]
            video_subtitles_path = os.path.join(SUBTITLES_PATH, f"{video_id}.vtt")
            video["subtitles"] = []

            language = get_language(video_subtitles_path)
            if language == "en":
                video["subtitles"] = extract_transcript(video_subtitles_path, None)

    ANNOTATIONS_PATH = os.path.join(PATH, "crosstask_release/annotations/")

    for task in library:
        for video in task["videos"]:
            video["annotations"] = []
            annotation_path = os.path.join(ANNOTATIONS_PATH, f"{task['task_id']}_{video['video_id']}.csv")
            if os.path.exists(annotation_path):
                with open(annotation_path) as f:
                    reader = csv.reader(f)
                    for row in reader:
                        video["annotations"].append({
                            "step": float(row[0]),
                            "start": float(row[1]),
                            "end": float(row[2]),
                        })
            else:
                print(f"No annotation found for {task['task_id']}_{video['video_id']}")

    ### label subtitles with step
    for task in library:
        for video in task["videos"]:
            annotated_subtitles = []
            for subtitle in video["subtitles"]:
                cur_step = None
                for annotation in video["annotations"]:
                    if subtitle["start"] >= annotation["start"] and subtitle["finish"] <= annotation["end"]:
                        cur_step = task["steps"][int(annotation["step"]) - 1]
                        break
                annotated_subtitles.append({
                    **subtitle,
                    "step": cur_step,
                })
            video["subtitles"] = annotated_subtitles

    ### restructure to be similar to the `dataset`

    ### save library as json
    with open(library_path, "w") as f:
        json.dump(library, f, indent=4)


def get_dataset_cross_task(task):
    """
    return dataset with the given task with a structure similar to the `dataset`
    """
    library = library_cross_task()
    dataset = []
    for _task in library:
        if _task["task_name"] == task:
            for video in _task["videos"]:
                content = ""
                transcript = []
                for subtitle in video["subtitles"]:
                    content += f"{subtitle['text']} "
                    transcript.append({
                        "text": subtitle['text'],
                        "start": subtitle['start'],
                        "end": subtitle['finish'],
                    })
                dataset.append({
                    "id": video["video_id"],
                    "url": video["video_url"],
                    "title": task,
                    "original_title": video["task"],
                    "content": content,
                    "transcript": transcript,
                    "steps": [],
                    "ipo": [],
                    "processed_ipos": [],
                })

    ### check if content is enough
    filtered_dataset = []
    for article in dataset:
        if len(article["content"]) < 100:
            continue
        filtered_dataset.append(article)
    dataset = filtered_dataset

    return dataset

def preprocess_cross_task(task, dummy=""):
    dataset_filepath = f"{FRAMEWORK_PATH}{task.replace(' ', '_').lower()}_{dummy}.json"
    if os.path.exists(dataset_filepath):
        with open(dataset_filepath) as f:
            dataset = json.load(f)
        return dataset


    dataset = get_dataset_cross_task(task)
    print(f"Dataset for {task}: {len(dataset)}")

    with open(dataset_filepath, "w") as f:
        json.dump(dataset, f, indent=4)
    return dataset

def preprocess_custom_dataset(task, dummy=""):
    dataset_filepath = f"{FRAMEWORK_PATH}{task.replace(' ', '_').lower()}_{dummy}.json"
    if os.path.exists(dataset_filepath):
        with open(dataset_filepath) as f:
            dataset = json.load(f)
        return dataset
    
    custom_tasks_path = "./static/datasets/custom-dataset/videos_tasks_per_category.json"
    
    with open(custom_tasks_path) as f:
        custom_tasks = json.load(f)

    videos = []
    category = None
    
    for _category in custom_tasks:
    
        for task_info in custom_tasks[_category]:
            if len(task_info["videos"]) < MIN_VIDEOS:
                continue
            _task = task_info["task_details"]["title"]
    
            if _task == task:
                videos.extend(task_info["videos"])
                category = _category
    

    if category is None:
        raise ValueError(f"Task {task} not found in any category")

    dataset = []
    for video in videos:
        content = ""
        for subtitle in video["transcript"]:
            content += f"{subtitle['text']} "
        dataset.append({
            "id": video["id"],
            "url": "https://www.youtube.com/watch?v=" + video["id"],
            "title": task,
            "original_title": video["title"],
            "category": category,
            "content": content,
            "transcript": video["transcript"],
            "steps": [],
            "ipo": [],
            "processed_ipos": [],
        })

    ### check if content is enough
    filtered_dataset = []
    for article in dataset:
        if len(article["content"]) < 100:
            continue
        filtered_dataset.append(article)
    dataset = filtered_dataset

    print(f"Dataset for {task}: {len(dataset)}")

    with open(dataset_filepath, "w") as f:
        json.dump(dataset, f, indent=4)
    return dataset    

### Describing Scope (attempt 1)

In [13]:
"""
    {label_structure} = {
        "title": {label title},
        "definition": {label_definition},
        "examples": [
            {
                "context": {text surrounding the content + content},
                "content": {content_text},
            }
        ]
    }
    {schema_structure} = {
        "schema": {facet title},
        "schema_plural": {facet plural},
        "definition": {definition of the schema/facet},
        
        "codebook_guidelines": [
            {guidelines for the LLM to generate the codebook}
            ...
        ],
        
        ### for labeling
        "labels": [
            {label_structure}
            ...
        ]
    }

    {labeled_dataset_structure} = {
        "url": {video url}
        "title": {task title},
        "original_title": {original video title},
        "content": {concatinated transcript},
        "transcript": [
            {
                "text": {text},
                "start": {start},
                "end": {end},
            },
            ...
        ],
        "pieces": {
            {piece_id}: {
                "piece_id": {piece_id},
                "unit_id": {unit_id},
                "content": {content of the piece},
                "content_type": {type of the content: Overview, Method, Explanation, Supplementary},
                "start": {start of the content},
                "end": {end of the content},
                "labels": {
                    {schema title}: [{label title}, ...], ### number of runs
                    ...
                }
            },
            ...
        }
    }

    approach_1_results = {
        "information_units": {
            {unit_id}: {
                "unit_id": {unit_id},
                "content": {content of the information unit},
                "content_type": {type of the content: Overview, Method, Explanation, Supplementary},
                "instances": [{id of the piece - piece_id}, ...],
            }
        },
        "context_schema": {
            {schema_name}: {schema_structure},
            ...
        },
        "facet_candidates": {
            {schema_name}: {schema_structure},
            ...
        },
        "labeled_dataset": [
            {labeled_dataset_structure},
            ...
        ]
    }
"""

'\n    {label_structure} = {\n        "title": {label title},\n        "definition": {label_definition},\n        "examples": [\n            {\n                "context": {text surrounding the content + content},\n                "content": {content_text},\n            }\n        ]\n    }\n    {schema_structure} = {\n        "schema": {facet title},\n        "schema_plural": {facet plural},\n        "definition": {definition of the schema/facet},\n        \n        "codebook_guidelines": [\n            {guidelines for the LLM to generate the codebook}\n            ...\n        ],\n        \n        ### for labeling\n        "labels": [\n            {label_structure}\n            ...\n        ]\n    }\n\n    {labeled_dataset_structure} = {\n        "url": {video url}\n        "title": {task title},\n        "original_title": {original video title},\n        "content": {concatinated transcript},\n        "transcript": [\n            {\n                "text": {text},\n                "st

In [14]:
def get_schemaname(schema):
    return "context_" + schema["schema"].replace(" ", "_").lower()

def get_cell_to_units(context_schema, dataset, important_piece_types):
    def get_label(piece, schema_name):
        if schema_name not in piece["labels"]:
            return ""
        if len(piece["labels"][schema_name]) == 0:
            return ""
        ### TODO: return the last label for now, later need to be adjusted
        return piece["labels"][schema_name][-1]

    cell_to_units = {}
    relevant_units = set()
    for video in dataset:
        for piece in video["pieces"]:
            if piece["content_type"] not in important_piece_types:
                continue
            cell_id = f"<{piece['content_type']}>"
            for schema in context_schema:
                cell_id += f"<{get_label(piece, get_schemaname(schema))}>"
            
            if cell_id not in cell_to_units:
                cell_to_units[cell_id] = {}
            
            if piece["unit_id"] not in cell_to_units[cell_id]:
                cell_to_units[cell_id][piece["unit_id"]] = []
            cell_to_units[cell_id][piece["unit_id"]].append(piece)
            relevant_units.add(piece["unit_id"])

    sparsity = len(important_piece_types)
    for schema in context_schema:
        sparsity *= len(schema["labels"]) + 1
    sparsity = len(cell_to_units) / sparsity * 100
    print("Sparsity %: ", sparsity)
    
    return cell_to_units, len(relevant_units)

In [15]:
from helpers.bert import bert_embedding, clustering_custom
from prompts.stupid_experiment_3 import form_information_units

def build_information_units_v0(task, dataset, information_unit_similarity_threshold=0.8):
    
    taskname = task.replace(" ", "_").lower()
    parent_path = os.path.join(FRAMEWORK_PATH, f'{taskname}')
    if not os.path.exists(parent_path):
        os.makedirs(parent_path)

    path = os.path.join(parent_path, "information_units_v0.json")
    if os.path.exists(path):
        with open(path) as f:
            dataset = json.load(f)
    
    for video_idx, video in enumerate(dataset):
        if "pieces" in video:
            pieces = video["pieces"]
        else:
            ### forming the information units (conceptually should be easily redefinable)
            pieces = form_information_units(task, video['transcript'])
            video['pieces'] = []
            for i, piece in enumerate(pieces):
                video['pieces'].append({
                    "piece_id": f"piece_{video_idx}_{i}",
                    **piece,
                    "labels": {},
                })

    ### TODO: maybe cluster for each type of information separately?
    all_pieces = []
    for video_idx, video in enumerate(dataset):
        for i, piece in enumerate(video['pieces']):
            piece["piece_id"] = f"piece_{video_idx}_{i}"
            all_pieces.append(piece)

    #### cluster similar pieces in `all_pieces`
    information_units = {}

    unit_labels = clustering_custom([piece["content"] for piece in all_pieces], information_unit_similarity_threshold)
    for i, piece in enumerate(all_pieces):
        cur_unit_id = f"unit_{unit_labels[i]}"
        piece["unit_id"] = cur_unit_id
        if cur_unit_id not in information_units:
            ### first piece is the representative of the cluster (IU)
            information_units[cur_unit_id] = {
                "content": piece["content"],
                "content_type": piece["content_type"],
                "instances": [piece["piece_id"]],
            }
        else:
            information_units[cur_unit_id]["instances"].append(piece["piece_id"])

    with open(path, "w") as f:
        json.dump(dataset, f, indent=4)

    return dataset, information_units

from prompts.stupid_experiment_3 import form_context_codebook, label_transcript_pieces

def build_codebook_v0(task, dataset, schema):
    """
    similar to VideoMix --> iteratively build the codebook
    TODO: may need to restrict the number of videos considered
    """
    # schema_name = get_schemaname(schema)
    # taskname = task.replace(" ", "_").lower()

    # parent_path = os.path.join(FRAMEWORK_PATH, f'{taskname}')
    # if not os.path.exists(parent_path):
    #     os.makedirs(parent_path)

    # path = os.path.join(parent_path, f"{schema_name}_v0.json")
    # if os.path.exists(path):
    #     with open(path) as f:
    #         schema = json.load(f)
    #         return schema

    ### iteratively build the context schema
    for video in dataset:
        new_labels = form_context_codebook(task, video["transcript"], schema)
        schema["labels"] = new_labels

    # with open(path, "w") as f:
    #     json.dump(schema, f, indent=4)

    return schema

def label_based_on_codebook_v0(task, dataset, schema):
    """
    label the dataset based on the codebook
    TODO: label multiple times per video
    """
    
    schema_name = get_schemaname(schema)
    # taskname = task.replace(" ", "_").lower()

    # parent_path = os.path.join(FRAMEWORK_PATH, f'{taskname}')
    # if not os.path.exists(parent_path):
    #     os.makedirs(parent_path)
    
    # path = os.path.join(parent_path, f"labeling_{schema_name}_v0.json")
    # if os.path.exists(path):
    #     with open(path) as f:
    #         dataset = json.load(f)
    #         return dataset

    for video in dataset:
        if "pieces" in video:
            if len(video["pieces"]) == 0:
                continue
            if schema_name in video["pieces"][0]["labels"]:
                ### skip if already labeled across `schema`
                continue
        else:
            print(f"WARNING: No pieces found for video {video['title']}")
            continue

        labeled_pieces = label_transcript_pieces(task, video["pieces"], schema)

        if len(labeled_pieces) != len(video["pieces"]):
            print(f"STRONG WARNING: {len(labeled_pieces)} != {len(video['pieces'])}")
        for piece_idx, piece in enumerate(video["pieces"]):
            if schema_name not in piece["labels"]:
                piece["labels"][schema_name] = []
            piece["labels"][schema_name].append(labeled_pieces[piece_idx])
    
    # if not os.path.exists(path):
    #     with open(path, "w") as f:
    #         json.dump(dataset, f, indent=4)

    return dataset

from prompts.stupid_experiment_3 import form_facet_candidates

def build_facet_candidates_v0(task, cell_to_units, facet_candidates, include_cells=10):
    """
    VideoMix-like approach --> go over the biggest cells and identify facet candidates beyond what we have and try to update the list? combine similar candidates as much as possible?
    """

    cell_ids = list(cell_to_units.keys())

    cell_ids.sort(key=lambda x: len(cell_to_units[x]), reverse=True)
    
    print("[OUTPUT]: Most common cells:")
    print([len(cell_to_units[cell_id]) for cell_id in cell_ids])
    
    cell_ids = cell_ids[:include_cells]

    for cell_id in cell_ids:
        if len(cell_to_units[cell_id]) == 0:
            break

        combined_pieces = []
        for unit_id in cell_to_units[cell_id]:
            combined_pieces.extend(cell_to_units[cell_id][unit_id])
        
        facet_candidates = form_facet_candidates(task, combined_pieces, facet_candidates)

    return facet_candidates

In [16]:
import math

def calc_discriminativeness(cell_to_units, relevant_units_count):
    ### TODO: adjust by noise
    base = 2

    if relevant_units_count == 0:
        return 0

    n_c_list = []

    total_entropy = 0    
    for cell_id in cell_to_units:
        n_c = len(cell_to_units[cell_id])

        if n_c == 0:
            continue

        p_c = float(n_c) / relevant_units_count
        total_entropy += p_c * math.log(n_c, base)

        n_c_list.append(n_c)

    check_entropy = math.log(relevant_units_count, base)
    for cell_id in cell_to_units:
        n_c = len(cell_to_units[cell_id])

        if n_c == 0:
            continue

        p_c = float(n_c) / relevant_units_count
        check_entropy += p_c * math.log(p_c, base)
    
    print("Entropy check: ", total_entropy, check_entropy, relevant_units_count, n_c_list)

    return total_entropy

def calc_compactness(context_schema, initial_labels):
    ### TODO: check if we prefer fewer facets or not
    ### need to blend the information types into context_schema
    total_labels = initial_labels-1
    for facet in context_schema:
        total_labels += len(facet["labels"]) ### because there is "empty" label

    return total_labels

def relative_improvement(d1, c1, d2, c2):
    compactness_weight = 0.001
    ## TODO: adjust by noise
    o1 = d1 + c1 * compactness_weight
    o2 = d2 + c2 * compactness_weight
    return o1-o2

def macro_pruning(context_schema, labeled_dataset, important_piece_types, threshold):
    if len(context_schema) <= 1:
        return context_schema
    
    cell_to_units, relevant_units_count = get_cell_to_units(context_schema, labeled_dataset, important_piece_types)
    ### macroprune the context schema by removing the least discriminative facet
    d_best = calc_discriminativeness(cell_to_units, relevant_units_count)
    c_best = calc_compactness(context_schema, len(important_piece_types))
    
    o_smallest = float("inf")
    facet_to_remove = None
    for i, _ in enumerate(context_schema):
        cur_context_schema = context_schema[:i] + context_schema[i+1:]
        cur_cell_to_units, cur_relevant_units_count = get_cell_to_units(cur_context_schema, labeled_dataset, important_piece_types)
        d = calc_discriminativeness(cur_cell_to_units, cur_relevant_units_count)
        c = calc_compactness(cur_context_schema, len(important_piece_types))
        
        o = relative_improvement(d, c, d_best, c_best)
        if o_smallest > o:
            o_smallest = o
            facet_to_remove = i
    if facet_to_remove is None or o_smallest > threshold:
        return context_schema
    return context_schema[:facet_to_remove] + context_schema[facet_to_remove+1:]

def update_facet_candidates(
    task, cell_to_units, facet_candidates, include_cells
):
    """
    Update the facet candidates and the labeled dataset.
    """

    if include_cells == 0:
        return facet_candidates

    taskname = task.replace(" ", "_").lower()
    parent_path = os.path.join(FRAMEWORK_PATH, f'{taskname}')
    if not os.path.exists(parent_path):
        os.makedirs(parent_path)
    
    path = os.path.join(parent_path, f"facet_candidates_v0.json")
    if os.path.exists(path):
        with open(path) as f:
            return json.load(f)

    facet_candidates = build_facet_candidates_v0(task, cell_to_units, facet_candidates, include_cells)

    with open(path, "w") as f:
        json.dump(facet_candidates, f, indent=4)

    return facet_candidates

def update_facet_labels(task, labeled_dataset, facet_candidates, include_cells):
    if include_cells == 0:
        return facet_candidates

    taskname = task.replace(" ", "_").lower()
    parent_path = os.path.join(FRAMEWORK_PATH, f'{taskname}')
    if not os.path.exists(parent_path):
        os.makedirs(parent_path)

    path = os.path.join(parent_path, f"facet_labels_v0.json")
    if os.path.exists(path):
        with open(path) as f:
            return json.load(f)

    updated_candidates = []
    for schema in facet_candidates:
        updated_candidates.append(
            build_codebook_v0(task, labeled_dataset, schema)
        )

    with open(path, "w") as f:
        json.dump(updated_candidates, f, indent=4)

    return updated_candidates

def update_labeled_dataset(task, labeled_dataset, facet_candidates):
    
    taskname = task.replace(" ", "_").lower()
    parent_path = os.path.join(FRAMEWORK_PATH, f'{taskname}')
    if not os.path.exists(parent_path):
        os.makedirs(parent_path)
    
    path = os.path.join(parent_path, f"labeled_dataset_v0.json")
    if os.path.exists(path):
        with open(path) as f:
            return json.load(f)

    ### label according to candidates
    for schema in facet_candidates:
        ### TODO: maybe explicitly ask to skip the unimportant types?
        labeled_dataset = label_based_on_codebook_v0(task, labeled_dataset, schema)

    with open(path, "w") as f:
        json.dump(labeled_dataset, f, indent=4)

    return labeled_dataset

In [ ]:
def process_videos_approach_1(task, dataset, important_piece_types):
    ### constants
    max_iterations = 100
    pruning_interval = 5
    pruning_threshold = 1
    max_macro_pruning_len = 2
    skip_pruning = True
    stopping_delta_threshold = 0.1
    include_cells = 10
    information_unit_similarity_threshold=0.9

    ### build the `information units`
    labeled_dataset, information_units = build_information_units_v0(task, dataset, information_unit_similarity_threshold)

    ### Greedy Algorithm for constructing the context schema:

    facet_candidates = [ ### start with subgoal & step
        # {
        #     "schema": "subgoal",
        #     "schema_plural": "subgoals",
        #     "definition": "an intermediate objective or subtask that helps in achieving the final goal",
        #     "codebook_guidelines": [
        #         "{schema_plural} should be high-level and concise.",
        #         "Base each {schema} on meaningful intermediate outcome/result.",
        #         "Exclude any {schema_plural} unrelated to the core task, such as introductions, conclusions, or general commentary."
        #     ],
        #     "labels": [],
        # },
        {
            "schema": "step",
            "schema_plural": "steps",
            "definition": "key high-level steps involved in the task",
            "codebook_guidelines": [
                "{schema_plural} should be high-level and concise.",
                "Base each {schema} on an intermediate outcome with tangible results (e.g., \"Make Dough\", \"Grill Steak\"), instead of individual actions (e.g., \"Add Flour\", \"Turn on Grill\").",
                "Avoid using specific ingredients in the {schema} name (e.g., \"Add Tomato Paste\"). Instead, focus on the purpose of the {schema} (e.g., \"Make Sauce\" instead of \"Add Tomato Paste\").",
                "Group together related low-level actions into a single high-level {schema}. (e.g., combine \"Add Salt\" and \"Add Lime\" into \"Make Sauce\").",
                "A {schema} must span multiple transcript sentences, not just a single sentence. It should be high-level enough.",
                "Use a concise \"verb + object\" format to describe each {schema}, containing only one verb (e.g., \"Boil Potatoes\").",
                "Exclude any {schema_plural} unrelated to the core task, such as introductions, conclusions, or general commentary."
            ],
            "labels": [
                {
                    "title": "Ingredient Preparation",
                    "definition": "Stage of preparing and measuring ingredients before mixing",
                    "examples": [],
                },
                {
                    "title": "Mixing",
                    "definition": "Stage of combining wet and dry ingredients into a uniform batter",
                    "examples": [],
                }
            ],
        },
    ]
    context_schema = []

    iteration_insights = []
    iterations = 0
    
    base_cell_to_units, base_relevant_units_count = get_cell_to_units(context_schema, labeled_dataset, important_piece_types)
    base_d = calc_discriminativeness(base_cell_to_units, base_relevant_units_count)
    base_c = calc_compactness(context_schema, len(important_piece_types))

    while True:

        iterations += 1
        if iterations > max_iterations:
            print("WARNING: Maximum number of iterations reached")
            break

        ### run pruning every `pruning_interval` iterations
        if iterations % pruning_interval == 0 and not skip_pruning:
            original_length = len(context_schema)
            while len(context_schema) > 1 and len(context_schema) > original_length - max_macro_pruning_len:
                new_context_schema = macro_pruning(
                    context_schema, labeled_dataset, important_piece_types, pruning_threshold
                )
                if len(new_context_schema) < len(context_schema):
                    context_schema = new_context_schema
                    continue
                else:
                    break   
            iteration_insights.append({
                "type": "macro_pruning",
                "iteration": iterations,
                "original_length": original_length,
                "new_length": len(context_schema),
            })

        cell_to_units, relevant_units_count = get_cell_to_units(context_schema, labeled_dataset, important_piece_types)
        # iteration_insights.append({
        #     "type": "get_cell_to_units",
        #     "iteration": iterations,
        #     "cell_to_units": cell_to_units,
        # })

        ### update the facet candidates
        facet_candidates = update_facet_candidates(task, cell_to_units, facet_candidates, include_cells)

        return {} ## testing

        ### update the facet labels
        facet_candidates = update_facet_labels(task, labeled_dataset, facet_candidates, include_cells)

        ### TODO: remove after testing
        include_cells = 0
        
        labeled_dataset = update_labeled_dataset(task, labeled_dataset, facet_candidates)
        
        if len(facet_candidates) == 0:
            print("WARNING: No facet candidates left")
            break    

        prev_d = calc_discriminativeness(cell_to_units, relevant_units_count)
        prev_c = calc_compactness(context_schema, len(important_piece_types))
        o_biggest = -float("inf")
        best_c = 1
        best_d = prev_d
        best_facet_toadd = None
        for i, facet_candidate in enumerate(facet_candidates):
            ### get the labeling by the context schema
            candidate_context_schema = context_schema + [facet_candidate]
            candidate_cell_to_units, candidate_relevant_units_count = get_cell_to_units(candidate_context_schema, labeled_dataset, important_piece_types)
            ### calculate the discriminative and compactness
            d = calc_discriminativeness(candidate_cell_to_units, candidate_relevant_units_count)
            c = calc_compactness(candidate_context_schema, len(important_piece_types))
            o = relative_improvement(prev_d, prev_c, d, c)
            iteration_insights.append({
                "type": "relative_improvement",
                "schema": facet_candidate["schema"],
                "iteration": iterations,
                "prev_d": prev_d,
                "prev_c": prev_c,
                "d": d,
                "c": c,
                "improvement": o,
                "absolute_improvement": relative_improvement(base_d, base_c, d, c),
            })
            if o > o_biggest:
                o_biggest = o
                best_d = d
                best_c = c
                best_facet_toadd = i

        iteration_insights.append({
            "type": "add_facet",
            "iteration": iterations,
            "prev_d": prev_d,
            "prev_c": prev_c,
            "best_d": best_d,
            "best_c": best_c,
            "improvement": o_biggest,
            "best_facet_toadd": best_facet_toadd,
            "best_facet": (None if best_facet_toadd is None else facet_candidates[best_facet_toadd]),
        })
        if o_biggest < stopping_delta_threshold:
            print("WARNING: Adding a facet didn't improve the objective `significantly`", o_biggest, stopping_delta_threshold)
            break

        if best_facet_toadd is None:
            print("WARNING: No best facet found / Seem to have converged")
            break
        context_schema.append(facet_candidates[best_facet_toadd])
        facet_candidates = facet_candidates[:best_facet_toadd] + facet_candidates[best_facet_toadd+1:]

    print("Completed in {} iterations".format(iterations))

    approach_1_results = {
        "information_units": information_units,
        "context_schema": context_schema,
        "facet_candidates": facet_candidates,
        "labeled_dataset": labeled_dataset,
        "cell_to_units": cell_to_units,
    }

    for insight in iteration_insights:
        print(json.dumps(insight, indent=4))

    ### save the results
    taskname = task.replace(" ", "_").lower()
    parent_path = os.path.join(FRAMEWORK_PATH, f'{taskname}')
    if not os.path.exists(parent_path):
        os.makedirs(parent_path)

    with open(os.path.join(parent_path, f"approach_1_results.json"), "w") as f:
        json.dump(approach_1_results, f, indent=4)

    return approach_1_results

In [18]:
def process_videos_approach_2(
    task,
    dataset,
    important_piece_types,
    beam_width=3,
    max_facets=5,
    iterations_for_facets_update=5,
):
    """
    Beam-search over facet subsets using relative_improvement against the
    empty-schema baseline as the objective. Periodically refresh facet candidates
    and labels every `iterations_for_facets_update` depths while keeping
    include_cells active.

    score = relative_improvement(base_d, base_c, d, c)
    """

    include_cells=10
    information_unit_similarity_threshold=0.9

    # Step 1: Build information units and initialize
    labeled_dataset, information_units = build_information_units_v0(
        task, dataset, information_unit_similarity_threshold
    )

    # Seed candidates
    facet_candidates = [
        {
            "schema": "step",
            "schema_plural": "steps",
            "definition": "key high-level steps involved in the task",
            "codebook_guidelines": [
                "{schema_plural} should be high-level and concise.",
                "Base each {schema} on an intermediate outcome with tangible results (e.g., \"Make Dough\", \"Grill Steak\"), instead of individual actions (e.g., \"Add Flour\", \"Turn on Grill\").",
                "Avoid using specific ingredients in the {schema} name (e.g., \"Add Tomato Paste\"). Instead, focus on the purpose of the {schema} (e.g., \"Make Sauce\" instead of \"Add Tomato Paste\").",
                "Group together related low-level actions into a single high-level {schema}. (e.g., combine \"Add Salt\" and \"Add Lime\" into \"Make Sauce\").",
                "A {schema} must span multiple transcript sentences, not just a single sentence. It should be high-level enough.",
                "Use a concise \"verb + object\" format to describe each {schema}, containing only one verb (e.g., \"Boil Potatoes\").",
                "Exclude any {schema_plural} unrelated to the core task, such as introductions, conclusions, or general commentary.",
            ],
            "labels": [],
        },
    ]

    # Initial candidate discovery and labeling
    base_cell_map = get_cell_to_units([], labeled_dataset, important_piece_types)
    facet_candidates = update_facet_candidates(task, base_cell_map, facet_candidates, include_cells)
    facet_candidates = update_facet_labels(task, labeled_dataset, facet_candidates, include_cells)
    labeled_dataset = update_labeled_dataset(task, labeled_dataset, facet_candidates)

    # Baseline for relative improvement
    base_d = calc_discriminativeness(base_cell_map)
    base_c = calc_compactness([])

    def compute_objective(context_schema):
        cell_map = get_cell_to_units(context_schema, labeled_dataset, important_piece_types)
        d = calc_discriminativeness(cell_map)
        c = calc_compactness(context_schema)
        score = relative_improvement(base_d, base_c, d, c)
        return score, d, c

    # Beam init
    candidate_indices = list(range(len(facet_candidates)))
    initial_score, initial_d, initial_c = compute_objective([])
    beam = [([], initial_score, initial_d, initial_c, set(candidate_indices))]
    best_overall = ([], initial_score, initial_d, initial_c)
    iteration_insights = []

    ### TODO: remove to enable periodic facet refresh
    include_cells = 0

    # Expand search
    for depth in range(1, max_facets + 1):
        next_beam = []
        for context_schema, score, d, c, remaining in beam:
            if not remaining:
                continue
            for idx in remaining:
                new_schema = context_schema + [facet_candidates[idx]]
                new_score, new_d, new_c = compute_objective(new_schema)
                new_remaining = set(remaining)
                if idx in new_remaining:
                    new_remaining.remove(idx)
                next_beam.append((new_schema, new_score, new_d, new_c, new_remaining))
                iteration_insights.append({
                    "type": "beam_expand",
                    "depth": depth,
                    "added_schema": facet_candidates[idx]["schema"],
                    "score": new_score,
                    "d": new_d,
                    "c": new_c,
                })
                if new_score > best_overall[1]:
                    best_overall = (new_schema, new_score, new_d, new_c)
        if not next_beam:
            break

        # Keep top-K
        next_beam.sort(key=lambda x: x[1], reverse=True)
        beam = next_beam[:beam_width]

        # Track layer summary
        layer_best = beam[0]
        iteration_insights.append({
            "type": "beam_layer",
            "depth": depth,
            "layer_best_score": layer_best[1],
            "layer_best_d": layer_best[2],
            "layer_best_c": layer_best[3],
            "global_best_score": best_overall[1],
            "global_best_d": best_overall[2],
            "global_best_c": best_overall[3],
        })

    # Output
    context_schema_best, best_score, best_d, best_c = best_overall
    approach_2_results = {
        "information_units": information_units,
        "context_schema": context_schema_best,
        "facet_candidates": facet_candidates,
        "labeled_dataset": labeled_dataset,
        "objective": {
            "score": best_score,
            "discriminativeness": best_d,
            "compactness": best_c,
            "baseline": {"d": base_d, "c": base_c},
        },
        "beam_width": beam_width,
        "max_facets": max_facets,
        "iterations_for_facets_update": iterations_for_facets_update,
        "include_cells": include_cells,
        "iteration_insights": iteration_insights,
    }

    # Persist
    taskname = task.replace(" ", "_").lower()
    parent_path = os.path.join(FRAMEWORK_PATH, f"{taskname}")
    if not os.path.exists(parent_path):
        os.makedirs(parent_path)
    with open(os.path.join(parent_path, f"approach_2_results.json"), "w") as f:
        json.dump(approach_2_results, f, indent=4)

    return approach_2_results

### Main

In [19]:
import random
import numpy as np
import json

def stats(dataset):
    non_zero_content_types = ["Method", "Description", "Explanation", "Supplementary"]

    per_content_type = {}
    for video in dataset:
        for i, piece in enumerate(video['baseline_results']):
            content_type = piece['content_type']
            if content_type not in per_content_type:
                per_content_type[content_type] = {
                    "count": 0,
                    "count_p_s": [],
                }
            per_content_type[content_type]["count"] += 1
            per_content_type[content_type]["count_p_s"].append(len(piece["procedure_segments"]))
            if content_type in non_zero_content_types and len(piece["procedure_segments"]) > 1 and content_type != "Method":
                ### print prev 2 and the next 2 pieces
                to_print = []
                for j in range(max(0, i - 2), min(len(video['baseline_results']), i + 3)):
                    if i == j:
                        to_print.append(f"[({video['baseline_results'][j]['content_type']}) {video['baseline_results'][j]['content']}]")
                        to_print.append(str(video['baseline_results'][j]['procedure_segments']))
                    else:
                        to_print.append(f"({video['baseline_results'][j]['content_type']}) {video['baseline_results'][j]['content']}")
                        to_print.append(str(video['baseline_results'][j]['procedure_segments']))
                # print("\n".join(to_print))
                # print("-" * 100)
    
    to_print = []
    for video in dataset:
        for piece in video['baseline_results']:
            if piece['content_type'] in non_zero_content_types and piece["content_type"] != "Method":
                print_str = ""
                print_str += piece['content_type'] + "\n"
                print_str += piece['content'] + "\n"
                print_str += str(piece['procedure_segments']) + "\n"
                print_str += str(piece['procedure_segments_clustered']) + "\n"
                print_str += "\n"
                to_print.append(print_str)
    
    ### shuffle to_print
    random.shuffle(to_print)
    print("\n".join(to_print))

In [20]:
MUFFIN_TASK = "Making Muffins"

"""
Make French Toast			10 steps / 272 videos
Make Irish Coffee			5 steps / 248 videos
Change a Tire				11 steps / 119 videos
Build (sim.) Floating Shelves		5 steps / 173 videos
"""
CROSS_TASK_TASKS = [
    "Change a Tire",
    "Build (sim.) Floating Shelves",
    "Make French Toast",
    "Make Irish Coffee",
]

CUSTOM_TASKS = [
    ### Food and Entertaining
    "How to Make a Sushi Roll",
    "How to Make Caramel Apples",
    "How to Make a Milkshake Without Ice Cream",
    "How to Grill Steak",
    "How to Make Scrambled Eggs in a Microwave",

    ### Home and Garden
    "How to Grow Hydrangea from Cuttings",
    "How to Grow a Pumpkin",
    "How to Clean Bathroom Tile",
    "How to Polish Stainless Steel",
    "How to Clean a Glass Top Stove",
    "How to Get Rid of a Wasp's Nest",

    # Holidays and Traditions
    "How to Plant a Living Christmas Tree",

    # Sports and Fitness
    "How to Wrap Your Hands for Boxing",
    "How to Catch Trout",

    # Arts and Entertainment
    "How to Make a Paper Hat",
]


def get_dataset(task):
    if task == MUFFIN_TASK:
        return get_dataset_muffins(task, "framework_raw")
    elif task in CROSS_TASK_TASKS:
        return preprocess_cross_task(task, "framework_raw")
    elif task in CUSTOM_TASKS:
        return preprocess_custom_dataset(task, "framework_raw")

def print_csv(dataset, ann_key="baseline_results", info_types=[]):
    filename = f"framework_raw_{'_'.join(ann_key.split(' '))}.csv"
    with open(filename, "w") as f:
        f.write("video_id,url,original_title,transcript_id,content_type,content,context\n")
        for v_id, video in enumerate(dataset):
            if ann_key not in video:
                continue
            for i, transcript in enumerate(video[ann_key]):
                cur_str = f"{v_id},{video['url'].strip()},{video['original_title'].strip()},{i},{transcript['content_type'].strip()},\"{transcript['content'].strip()}\","
                if "context_step" in transcript:
                    cur_str += f"\"{transcript['context_step'].strip()}\""
                else:
                    if transcript['content_type'] in info_types:
                        cur_str += "\"<empty>\""
                    else:
                        cur_str += "\"<not-assigned>\""
                cur_str += "\n"
                f.write(cur_str)

def main(task):
    dataset = get_dataset(task)

### Fragmentation Analysis

In [21]:
from helpers.bert import bert_embedding
import numpy as np
import matplotlib.pyplot as plt

def shannon_entropy(distribution):
    """
    Calculate the Shannon entropy of a distribution.
    """
    return -np.sum(distribution * np.log(distribution + 1e-10))

def trace_of_covariance(pieces):
    if len(pieces) == 0:
        return 0
    trace = 0
    mean = np.mean(pieces, axis=0)
    for i in range(len(pieces)):
        trace += (pieces[i] - mean) @ (pieces[i] - mean).T
    trace /= len(pieces)
    return trace

def compare_fragmentation(pieces, labels, label_vocab, distance_delta=0.01):
    """
    Check if global fragmentation is higher or lower than the labeled fragmentation.
    """
    similarity_matrix = np.zeros((len(pieces), len(pieces)))
    for i in range(len(pieces)):
        for j in range(i+1):
            similarity_matrix[i, j] = np.dot(pieces[i], pieces[j])
            similarity_matrix[j, i] = similarity_matrix[i, j]

    distance_buckets = np.arange(0, 1+distance_delta, distance_delta)
    
    global_distance_distribution = np.zeros(len(distance_buckets))
    for i in range(len(pieces)):
        for j in range(len(pieces)):
            if i == j:
                continue
            distance = similarity_matrix[i, j]
            bucket = 0
            for i in range(len(distance_buckets)):
                if distance >= distance_buckets[i]:
                    bucket = i
            global_distance_distribution[bucket] += 1
    
    ### print # of global distances
    print(f"# of global distances: {np.sum(global_distance_distribution)}")
    print(f"Trace of covariance: {trace_of_covariance(pieces)}")

    ### normalize global distance distribution
    global_distance_distribution = global_distance_distribution / np.sum(global_distance_distribution) * 100
    
    distance_distribution_per_label = {}

    for label in label_vocab:
        labeled_distance_distribution = np.zeros(len(distance_buckets))
        cur_pieces = []
        for i in range(len(pieces)):
            if labels[i] != label:
                continue
            cur_pieces.append(pieces[i])
            for j in range(len(pieces)):
                if labels[j] != label:
                    continue
                if i == j:
                    continue
                distance = similarity_matrix[i, j]
                bucket = 0
                for i in range(len(distance_buckets)):
                    if distance >= distance_buckets[i]:
                        bucket = i
                labeled_distance_distribution[bucket] += 1
        
        ### print # of labeled distances
        print(f"# of labeled distances for {label}: {np.sum(labeled_distance_distribution)}")
        print(f"Trace of covariance for {label}: {trace_of_covariance(cur_pieces)}")

        distance_distribution_per_label[label] = labeled_distance_distribution / np.sum(labeled_distance_distribution) * 100

    ### plot the distance distribution in the same plot
    plt.figure(figsize=(10, 5))
    plt.plot(distance_buckets * 100, global_distance_distribution, label="Global", linestyle="--")
    for label in label_vocab:
        plt.plot(distance_buckets * 100, distance_distribution_per_label[label], label=label)
    plt.legend()
    plt.show()

custom_labels = {
    0: "350F",
    1: "350F",
    2: "375F",
    3: "375F",
    4: "375F",
    5: "350F",
    6: "??",
    7: "??",
    8: "425F",
    9: "??",
    10: "425F",
}

def experiment_analysis(dataset):
    """
    Embed all information pieces in the dataset;
    """
    # content_types_to_include = ["Method", "Description", "Explanation", "Supplementary"]
    content_types_to_include = ["Method"]
    pieces = []
    labels = []
    label_vocab = []
    for idx, video in enumerate(dataset):
        label = custom_labels[idx]
        if label not in label_vocab:
            label_vocab.append(label)
        for piece in video['baseline_results']:
            if piece['content_type'] not in content_types_to_include:
                continue
            # label = piece['content_type']
            # if label not in label_vocab:
            #     label_vocab.append(label)
            pieces.append(piece['content'])
            labels.append(label)
    
    pieces = bert_embedding(pieces)
    compare_fragmentation(pieces, labels, label_vocab, 0.05)

#### output
```
# of global distances: 110555.0
Trace of covariance: 0.6744245930655941
# of labeled distances for 350F: 11341.0
Trace of covariance for 350F: 0.7017171226929282
# of labeled distances for 375F: 8099.0
Trace of covariance for 375F: 0.6654950787623723
# of labeled distances for ??: 2600.0
Trace of covariance for ??: 0.5681160431282193
# of labeled distances for 425F: 7224.0
Trace of covariance for 425F: 0.6282378954045913
```
![image](/home/bekzat/starlab/video-analysis/probe/server/static/results/frag-analysis-1.png)

### OUTPUT

In [22]:
import os

task = MUFFIN_TASK
# task = CUSTOM_TASKS[14]
# task = CROSS_TASK_TASKS[0]
# task = CROSS_TASK_TASKS[1]
# task = CUSTOM_TASKS[13]

### `Greeting`, `Overview`, `Method`, `Supplementary`, `Explanation`, `Description`, `Conclusion`, and `Miscellaneous`
important_types = ["Method", "Supplementary", "Explanation", "Description"]

dataset = get_dataset(task)

result = process_videos_approach_1(task, dataset[7:], important_types)

Sparsity %:  100.0
Entropy check:  5.093728905753884 5.093728905753884 68 [54, 8, 5, 1]
Sparsity %:  100.0
Entropy check:  5.093728905753884 5.093728905753884 68 [54, 8, 5, 1]
Sparsity %:  40.625
Entropy check:  3.0582358622576886 2.96871434988636 68 [6, 26, 6, 5, 2, 5, 3, 4, 3, 2, 4, 2, 1]
Sparsity %:  47.72727272727273
Entropy check:  2.2031626059001073 2.1136410935287793 68 [13, 7, 3, 2, 7, 7, 1, 4, 4, 4, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1]
Sparsity %:  46.42857142857143
Entropy check:  3.1161705683829433 3.026649056011614 68 [20, 17, 1, 3, 5, 5, 4, 4, 4, 2, 1, 2, 1]
Sparsity %:  44.44444444444444
Entropy check:  2.9062192491585543 2.727176224415897 68 [24, 8, 5, 5, 2, 4, 2, 1, 3, 5, 1, 2, 4, 1, 1, 2]
Sparsity %:  47.72727272727273
Entropy check:  2.2031626059001073 2.1136410935287793 68 [13, 7, 3, 2, 7, 7, 1, 4, 4, 4, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1]
Sparsity %:  9.943181818181818
Entropy check:  1.287143932872506 1.197622420501177 68 [1, 4, 5, 7, 3, 1, 3, 1, 1, 3, 2, 4, 2, 2, 2, 4, 2

### Test

In [15]:
important_types = ["Method", "Supplementary", "Explanation", "Description"]

dataset, information_units = build_information_units_v0(task, dataset, information_unit_similarity_threshold=0.8)

facet_candidates = [ ### start with subgoal & step
    {
        "schema": "subgoal",
        "schema_plural": "subgoals",
        "definition": "an intermediate objective or subtask that helps in achieving the final goal",
        "codebook_guidelines": [
            "Subgoals should be high-level and concise.",
            "Base each subgoal on meaningful intermediate outcome/result.",
            "Exclude any subgoals unrelated to the core task, such as introductions, conclusions, or general commentary."
        ],
        "labels": [],
    },
    {
        "schema": "step",
        "schema_plural": "steps",
        "definition": "key high-level steps involved in the task",
        "codebook_guidelines": [
            "Steps should be high-level and concise.",
            "Base each step on an intermediate outcome with tangible results (e.g., \"Make Dough\", \"Grill Steak\"), instead of individual actions (e.g., \"Add Flour\", \"Turn on Grill\").",
            "Avoid using specific ingredients in the step name (e.g., \"Add Tomato Paste\"). Instead, focus on the purpose of the step (e.g., \"Make Sauce\" instead of \"Add Tomato Paste\").",
            "Group together related low-level actions into a single high-level step. (e.g., combine \"Add Salt\" and \"Add Lime\" into \"Make Sauce\").",
            "A step must span multiple transcript sentences, not just a single sentence. It should be high-level enough.",
            "Use a concise \"verb + object\" format to describe each step, containing only one verb (e.g., \"Boil Potatoes\").",
            "Exclude any steps unrelated to the core task, such as introductions, conclusions, or general commentary."
        ],
        "labels": [],
    }
]

## only for step/subgoal
context_schema = []
cur_schema = build_codebook_v0(
    task, dataset, facet_candidates[1])
context_schema.append(cur_schema)

facet_candidates.remove(facet_candidates[1])

labeled_dataset = label_based_on_codebook_v0(task, dataset, context_schema[0])

cell_to_units = get_cell_to_units(context_schema, labeled_dataset, important_types)

updated_candidates = build_facet_candidates_v0(task, cell_to_units, facet_candidates, include_cells=2)

MESSAGES: [
  {
    "role": "system",
    "content": "\nYou are a helpful assistant who can understand and analyze tutorial videos."
  },
  {
    "role": "user",
    "content": "\nYou are analyzing a tutorial video for Making Muffins.\nFrom a tutorial-style transcript (recipe, SOP, repair guide, etc.) extract the steps involved in the task. A step is key high-level steps involved in the task.\n\nFollow these guidelines when extracting steps:\n- Steps should be high-level and concise.\n- Base each step on an intermediate outcome with tangible results (e.g., \"Make Dough\", \"Grill Steak\"), instead of individual actions (e.g., \"Add Flour\", \"Turn on Grill\").\n- Avoid using specific ingredients in the step name (e.g., \"Add Tomato Paste\"). Instead, focus on the purpose of the step (e.g., \"Make Sauce\" instead of \"Add Tomato Paste\").\n- Group together related low-level actions into a single high-level step. (e.g., combine \"Add Salt\" and \"Add Lime\" into \"Make Sauce\").\n- A ste